<a href="https://colab.research.google.com/github/2203A51461/2203A51461_AIML2024/blob/main/Assignment_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

preprocessing of data and dividing the data

In [1]:
# Import necessary libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from tensorflow.keras.datasets import imdb

# Load dataset (Using IMDB for text generation)
vocab_size = 10000
max_length = 100

# Load IMDB dataset
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=vocab_size)

# Pad sequences to ensure uniform input size
X_train = pad_sequences(X_train, maxlen=max_length)
X_test = pad_sequences(X_test, maxlen=max_length)

# Display dataset shape
print(f'Training data shape: {X_train.shape}')
print(f'Testing data shape: {X_test.shape}')


17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Training data shape: (25000, 100)
Testing data shape: (25000, 100)


build gru

In [2]:
# Define the GRU model
def build_gru_model(vocab_size, embedding_dim=64, input_length=max_length):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=input_length),
        tf.keras.layers.GRU(128, return_sequences=False),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Initialize and summarize the GRU model
gru_model = build_gru_model(vocab_size)
gru_model.summary()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru (GRU)                            │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

training gru

In [3]:
# Train the GRU model
gru_history = gru_model.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test), batch_size=64)


Epoch 1/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 127s 311ms/step - accuracy: 0.6123 - loss: 0.6431 - val_accuracy: 0.8336 - val_loss: 0.3738
Epoch 2/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 131s 284ms/step - accuracy: 0.8733 - loss: 0.3033 - val_accuracy: 0.8485 - val_loss: 0.3490
Epoch 3/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 141s 282ms/step - accuracy: 0.9201 - loss: 0.2142 - val_accuracy: 0.8204 - val_loss: 0.3971
Epoch 4/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 111s 284ms/step - accuracy: 0.9486 - loss: 0.1424 - val_accuracy: 0.8348 - val_loss: 0.4178
Epoch 5/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 146s 294ms/step - accuracy: 0.9678 - loss: 0.0941 - val_accuracy: 0.8356 - val_loss: 0.5456


txt generation using trained model

In [4]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Sample text data (used for training and fitting the tokenizer)
text_data = [
    "Once upon a time in a faraway land there lived a king",
    "The king had three sons who were brave and kind",
    "They went on adventures through forests and mountains"
]

# Initialize and fit the Tokenizer on the training text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_data)

# Parameters for text generation
seed_text = "Once upon a time"
next_words = 15  # Number of words to generate
max_length = 100  # Max length used during model training (adjust as per your model)

# Assuming `gru_model` is the pre-trained GRU model for text generation
for _ in range(next_words):
    # Convert the current seed text to a sequence of integers (tokens)
    token_list = tokenizer.texts_to_sequences([seed_text])[0]

    # Pad the sequence to the correct length (max_length - 1)
    token_list = pad_sequences([token_list], maxlen=max_length - 1, padding='pre')

    # Predict the next word's token using the GRU model
    predicted = np.argmax(gru_model.predict(token_list), axis=-1)

    # Find the word corresponding to the predicted token
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break

    # Append the predicted word to the seed text
    seed_text += " " + output_word

# Print the final generated text
print(seed_text)



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 235ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Once upon a time               


In [5]:
# Evaluate GRU model accuracy
gru_loss, gru_accuracy = gru_model.evaluate(X_test, y_test)
print(f"GRU Model Accuracy: {gru_accuracy * 100:.2f}%")


782/782 ━━━━━━━━━━━━━━━━━━━━ 21s 27ms/step - accuracy: 0.8324 - loss: 0.5620
GRU Model Accuracy: 83.56%


In [6]:
# Define the LSTM model
def build_lstm_model(vocab_size, embedding_dim=64, input_length=max_length):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=input_length),
        tf.keras.layers.LSTM(128, return_sequences=False),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Initialize and summarize the LSTM model
lstm_model = build_lstm_model(vocab_size)
lstm_model.summary()

# Train the LSTM model
lstm_history = lstm_model.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test), batch_size=64)

# Evaluate LSTM model accuracy
lstm_loss, lstm_accuracy = lstm_model.evaluate(X_test, y_test)
print(f"LSTM Model Accuracy: {lstm_accuracy * 100:.2f}%")


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 109s 271ms/step - accuracy: 0.6989 - loss: 0.5571 - val_accuracy: 0.8336 - val_loss: 0.3800
Epoch 2/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 108s 277ms/step - accuracy: 0.8889 - loss: 0.2817 - val_accuracy: 0.8468 - val_loss: 0.3598
Epoch 3/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 141s 275ms/step - accuracy: 0.9245 - loss: 0.1989 - val_accuracy: 0.8418 - val_loss: 0.4294
Epoch 4/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 145s 282ms/step - accuracy: 0.9471 - loss: 0.1433 - val_accuracy: 0.8282 - val_loss: 0.4845
Epoch 5/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 108s 278ms/step - accuracy: 0.9626 - loss: 0.1025 - val_accuracy: 0.8348 - val_loss: 0.5886
782/782 ━━━━━━━━━━━━━━━━━━━━ 39s 49ms/step - accuracy: 0.8345 - loss: 0.5956
LSTM Model Accuracy: 83.48%


In [7]:
print(f"GRU Model Accuracy: {gru_accuracy * 100:.2f}%")
print(f"LSTM Model Accuracy: {lstm_accuracy * 100:.2f}%")


GRU Model Accuracy: 83.56%
LSTM Model Accuracy: 83.48%
